In [1]:
import pandas as pd
from pymongo import MongoClient
import folium
import ipyleaflet as ipyl
import ipywidgets as widget

In [2]:
# Create instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# Assign database to variable
db = mongo['Bridges']
collection = db['PhillyBridges']
data = collection.find()
df = pd.DataFrame(list(data))
df

,_id,Bridge_ID,Latitude,Longitude,Year_Built,Bridge_Condition,Average Daily_Traffic,Deck_Width,Structure_Length,SqFt,Repair_Cost_Est,Replacement_Cost_Est,Address
0,66252a9b53c092ad819b9915,452997,40.103450,-74.961092,1990,Good,9729,13.0,18.3,237.90,"$74,700.60","$109,852.70","Dunksferry Road, Mechanicsville, 19020"
1,66252a9b53c092ad819b9916,466574,40.017092,-75.211428,1928,Fair,14596,19.2,32.9,631.68,"$198,347.52","$291,684.56","Ridge Avenue, Wissahickon, 19127"
2,66252a9b53c092ad819b9917,466580,39.963467,-75.186778,1964,Poor,27042,16.1,167.0,2688.70,"$844,251.80","$1,241,534.11","Spring Garden Street, Mantua, 19104"
3,66252a9b53c092ad819b9918,466581,39.963861,-75.185025,1964,Poor,27042,16.2,27.1,439.02,"$137,852.28","$202,721.88","2917, Spring Garden Street, Mantua, 19104"
4,66252a9b53c092ad819b9919,466617,40.035356,-75.208878,1957,Fair,28301,20.7,115.2,2384.64,"$748,776.96","$1,101,131.37","Henry Avenue, 19127"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,66252a9b53c092ad819b99ae,471580,39.958900,-75.170372,2017,Good,8800,19.8,29.9,592.02,"$185,894.28","$273,371.16","North 19th Street, Center City, 19132"
154,66252a9b53c092ad819b99af,471996,39.959400,-75.173906,2018,Good,13020,20.4,39.0,795.60,"$249,818.40","$367,376.26","Vine Street Expressway, Center City, 19103"
155,66252a9b53c092ad819b99b0,471998,39.959500,-75.175500,2018,Good,12400,25.5,32.9,838.95,"$263,430.30","$387,393.55","North 22nd Street, Center City, 19103"
156,66252a9b53c092ad819b99b1,472287,39.960194,-75.159694,1892,Poor,150,4.8,17.1,82.08,"$25,773.12","$37,901.26","Noble Street, Chinatown, Center City, 19123"


In [4]:
# Create map of Philadelphia
myMap = folium.Map(location = (39.952583, -75.165222), tiles = "OpenStreetMap")
myMap

In [5]:
# Add another tile layer for visualization
folium.TileLayer("cartodbpositron").add_to(myMap)

In [6]:
# Create function to color markers
condition = df["Bridge_Condition"]
def get_marker_color(condition):
    if condition == "Good":
        return "green"
    elif condition == "Fair":
        return "orange"
    elif condition == "Poor":
        return "darkred"
    else:
        return "gray"

# Create separate feature groups for each marker type
good_group = folium.FeatureGroup(name="Good (G)")
fair_group = folium.FeatureGroup(name="Fair (F)")
poor_group = folium.FeatureGroup(name="Poor (P)")

# Create Markers 
for index, row in df.iterrows():
    # Assign the variables
    lat = row["Latitude"]
    long = row["Longitude"]
    condition = row["Bridge_Condition"]
    address = row["Address"]
    repair_cost = row["Repair_Cost_Est"]
    replace_cost = row["Replacement_Cost_Est"]
    
    # Marker Coloring
    marker_color = get_marker_color(condition)
    text = f"Address: {address}<br>Latitude: {lat} <br>Longitude: {long} <br>Condition: {condition} <br>Estimated Repair ($): {repair_cost} <br>Estimated Replacement ($): {replace_cost}"
    popup = folium.Popup(text, max_width = 300)
    marker = folium.Marker([lat, long], popup=popup, icon=folium.Icon(color=marker_color))

# Add markers to their respective group   
    if condition == "Good":
        marker.add_to(good_group)
    elif condition == "Fair":
        marker.add_to(fair_group)
    elif condition == "Poor":
        marker.add_to(poor_group)

# Add groups to map
good_group.add_to(myMap)
fair_group.add_to(myMap)
poor_group.add_to(myMap)

folium.LayerControl(collapsed=False).add_to(myMap)

myMap

In [7]:
def get_circle_color(value):
    if value < 25000:
        return 'green'
    elif value >= 25000 and value < 49999:
        return 'orange'
    else:
        return 'red'

#Create separate groups for each cirlce color

low_traffic = folium.FeatureGroup(name="Low Traiffic")
med_traffic = folium.FeatureGroup(name="Medium Traffic")
high_traffic = folium.FeatureGroup(name="High Traffic")

# Iterate over your DataFrame
for index, row in df.iterrows():
    # Get the value from your column
    value = row["Average Daily_Traffic"]

    # Determine the color based on the value ranges
    color = get_circle_color(value)

    # Assign radius
    if value < 25000:
        radius = 5
    elif value >= 25000 and value < 49999:
        radius = 10
    else:
        radius = 15
        
    # Add a circle to the map with the determined color
    circle = folium.Circle(location=[row["Latitude"], row["Longitude"]], 
                           radius=radius, 
                           color=color, 
                           fill_color=color)    
    
    if value < 25000:
        circle.add_to(low_traffic)
    elif value >= 25000 and value < 50000:
        circle.add_to(med_traffic)
    else:
        circle.add_to(high_traffic)


# Add the three traffic level groups to the map      
low_traffic.add_to(myMap)
med_traffic.add_to(myMap)
high_traffic.add_to(myMap)


folium.LayerControl().add_to(myMap)  
    
    
myMap